In [1]:
import os
from openai import OpenAI
import json
from tqdm import tqdm
import time

In [2]:
input_json_list = list()
with open("webqsp_hit30.json", 'r') as file:
    lines = file.readlines()
    for line in tqdm(lines):
        line = line.strip()
        json_line = json.loads(line)
        input_json_list.append(json_line)

100%|██████████| 1541/1541 [00:00<00:00, 12427.56it/s]


In [3]:
new_question_list = list()
for piece_json in tqdm(input_json_list):
    all_paths = ""
    for elem in piece_json['top10_path']:
        if piece_json['top10_path'][elem][0]['priority'] < 30: 
        # print(piece_json['top10_path'][elem][0]['path'])
            all_paths += "{Path: " +  piece_json['top10_path'][elem][0]['path'] + "}\n"
    new_question_piece = {
        'Question': piece_json['question'] + '\n',
        'Relational Facts': all_paths,
        'Answer List': piece_json['answer'],
    }
    new_question_list.append(new_question_piece)
# print(new_question_piece)
print(len(new_question_list))
# print(all_paths)

100%|██████████| 1541/1541 [00:00<00:00, 62573.67it/s]

1541


In [6]:
input_question = new_question_list[1]
answer_list = input_question['Answer List']
question_piece = input_question['Question'] + input_question['Relational Facts']
print(question_piece)

what did james k polk do before he was president
{Path: James K. Polk --> government.politician.government_positions_held --> m.04j5sk8 --> government.government_position_held.office_position_or_title --> Governor of Tennessee}
{Path: James K. Polk --> government.politician.government_positions_held --> m.04j60kc --> government.government_office_or_title.office_holders_reverse --> United States Representative}
{Path: James K. Polk --> government.politician.government_positions_held --> m.0944j8_ --> government.government_office_or_title.office_holders_reverse --> Speaker of the United States House of Representatives}
{Path: James K. Polk --> government.government_position_held.office_holder_reverse --> m.04469y8 --> government.government_office_or_title.office_holders_reverse --> President dels Estats Units}



In [7]:
import requests
import re
count = 0
correct = 0
output_list = list()
i = 0
j = 0
for input_question in tqdm(new_question_list):
    answer_list = input_question['Answer List']
    question_piece = question_piece = input_question['Question'] + input_question['Relational Facts']

    headers = {
        "Content-Type": "application/json",
        "Authorization": "your API key"  
    }
    data = {
        "model": "gpt-3.5-turbo-0125",
        "messages": [
            {
                "role": "user",
                "content": f"{question_piece}" + '\n'
            },
            {
                "role": "user",
                "content": """Based on the relational facts provided, you are asked to return all answer entities directly."""
            }
        ],
        "temperature": 0,
    }

    try_again = 0
    while try_again < 5:
        response = requests.post("https://api.openai.com/v1/chat/completions", json=data, headers=headers)
        if response.status_code == 200:
            break
        try_again += 1
    if try_again == 5:
        print("api call failed!")
        continue

    prediction = response.json()['choices'][0]['message']['content']

    is_correct = False
    filtered_prediction = re.sub(r'[^a-zA-Z0-9\s]', '', prediction)
    for answer in answer_list:
        if '(' in answer and ')' in answer:
            answer = answer[:answer.index('(')]
        filtered_answer = re.sub(r'[^a-zA-Z0-9\s]', '', answer)
        if filtered_answer.lower() in filtered_prediction.lower():
            is_correct = True
            correct += 1
            break
        elif filtered_prediction.lower() in filtered_answer.lower():
            is_correct = True
            correct += 1
            break
        elif '{' in prediction and '}' in prediction:
            entity = re.sub(r'[^a-zA-Z0-9\s]', '', prediction[prediction.index('{')+1:prediction.index('}')])
            if entity.lower() in filtered_answer.lower() or filtered_answer.lower() in entity.lower():
                is_correct = True
                correct += 1
                break
        else:
            is_correct = False
    count += 1

    output_data = {}
    output_data['question'] = question_piece
    output_data['prediction'] = prediction
    output_data['answer_list'] = answer_list
    output_data['is_correct'] = is_correct
    output_data = json.dumps(output_data)
    output_list.append(output_data)

file = open('result.txt', 'w')
for elem in output_list:
    file.write(elem)
    file.write('\n')
file.close()
print(correct/count)


  0%|          | 0/1541 [00:00<?, ?it/s]

100%|██████████| 1541/1541 [37:16<00:00,  1.45s/it] 

0.8494484101232965
